In [ ]:
!kaggle datasets download -d tawsifurrahman/covid19-radiography-database

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import zipfile



In [2]:
import zipfile

# ZIP dosyasının yolu
zip_file_path = '/content/covid19-radiography-database.zip'

# Çıkartılacak hedef klasör
extracted_folder_path = '/content/'

# ZIP dosyasını aç
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

In [ ]:
import os
import shutil
import numpy as np
from sklearn.model_selection import train_test_split

data_dir = '/content/COVID-19_Radiography_Dataset'

train_dir = '/content/TrainTemp'
test_dir = '/content/Test'

classes = ['COVID', 'Normal', 'Viral Pneumonia']

os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Her sınıf için eğitim ve test dizinlerini oluştur
for cls in classes:
    os.makedirs(os.path.join(train_dir, cls), exist_ok=True)
    os.makedirs(os.path.join(test_dir, cls), exist_ok=True)

# Her sınıf için görsel dosyalarını eğitim ve test dizinlerine kopyala
for cls in classes:
    cls_dir = os.path.join(data_dir, cls, 'images')
    images = os.listdir(cls_dir)
    train_images, test_images = train_test_split(images, test_size=0.2, random_state=99)

    for img in train_images:
        src = os.path.join(cls_dir, img)
        dst = os.path.join(train_dir, cls, img)
        shutil.copy(src, dst)

    for img in test_images:
        src = os.path.join(cls_dir, img)
        dst = os.path.join(test_dir, cls, img)
        shutil.copy(src, dst)


In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

train_dir = '/content/TrainTemp'

train_data_dir = '/content/Train'
validation_data_dir = '/content/Validation'
test_data_dir = '/content/Test'  # Test veri setinin bulunduğu dizin

os.makedirs(train_data_dir, exist_ok=True)
os.makedirs(validation_data_dir, exist_ok=True)

classes = ['COVID', 'Normal', 'Viral Pneumonia']

# Her sınıf için eğitim ve doğrulama dizinlerini oluştur
for cls in classes:
    os.makedirs(os.path.join(train_data_dir, cls), exist_ok=True)
    os.makedirs(os.path.join(validation_data_dir, cls), exist_ok=True)

# Her sınıf için eğitim verisini eğitim ve doğrulama setlerine ayır
for cls in classes:
    cls_dir = os.path.join(train_dir, cls)
    train_images = os.listdir(cls_dir)
    train_images, validation_images = train_test_split(train_images, test_size=0.125, random_state=99)

    for img in train_images:
        src = os.path.join(cls_dir, img)
        dst = os.path.join(train_data_dir, cls, img)
        shutil.copy(src, dst)

    for img in validation_images:
        src = os.path.join(cls_dir, img)
        dst = os.path.join(validation_data_dir, cls, img)
        shutil.copy(src, dst)


import shutil

shutil.rmtree("/content/TrainTemp")

In [11]:

data_transforms = {
    'Train': transforms.Compose([
        transforms.RandomResizedCrop(299),  # 299x299 boyutuna göre kırpma
        transforms.ToTensor(),  # Tensor çevirme
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalizasyon
    ]),
    'Validation': transforms.Compose([
        transforms.Resize(320),  # Yeniden boyutlandırma
        transforms.CenterCrop(299),  # Ortadan kırpma
        transforms.ToTensor(),  # Tensor çevirme
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalizasyon
    ]),
    'Test': transforms.Compose([
        transforms.Resize(320),  # Yeniden boyutlandırma
        transforms.CenterCrop(299),  # Ortadan kırpma
        transforms.ToTensor(),  # Tensor çevirme
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalizasyon
    ]),
}

image_datasets = {
    'Train': datasets.ImageFolder(root=train_data_dir, transform=data_transforms['Train']),
    'Validation': datasets.ImageFolder(root=validation_data_dir, transform=data_transforms['Validation']),
    'Test': datasets.ImageFolder(root=test_data_dir, transform=data_transforms['Validation'])
}

dataloaders = {
    'Train': DataLoader(image_datasets['Train'], batch_size=32, shuffle=True),
    'Validation': DataLoader(image_datasets['Validation'], batch_size=32, shuffle=False),
    'Test': DataLoader(image_datasets['Test'], batch_size=32, shuffle=False)
}


In [12]:
import torch.nn as nn
import torchvision.models as models

# VGG19 modelini yükleme
inceptionv3 = models.inception_v3(pretrained=True)

# Son fully connected layer'ı değiştirme
num_ftrs = inceptionv3.fc.in_features
inceptionv3.fc = nn.Linear(num_ftrs, len(image_datasets['Train'].classes))

# Eğitim için CUDA kullanılabilirse GPU'ya taşıma ve DataParallel kullanarak sarmalama
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
inceptionv3 = inceptionv3.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth
100%|██████████| 104M/104M [00:01<00:00, 80.2MB/s] 


In [16]:
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
import copy
from tqdm import tqdm

# Optimizasyon algoritması ve kayıp fonksiyonu
optimizer = optim.SGD(inceptionv3.parameters(), lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss()

# Eğitim döngüsü
epochs = 3
best_model_wts = copy.deepcopy(inceptionv3.state_dict())
best_acc = 0.0

for epoch in range(epochs):
    print(f'Epoch {epoch+1}/{epochs}')
    print('-' * 10)

    for phase in ['Train', 'Validation']:
        if phase == 'Train':
            inceptionv3.train()
        else:
            inceptionv3.eval()

        running_loss = 0.0
        running_corrects = 0

        data_loader = tqdm(dataloaders[phase], desc=f'{phase} Phase', unit="batch", leave=True)
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'Train'):
                outputs = inceptionv3(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                if phase == 'Train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

            data_loader.set_postfix({'loss': loss.item()})

        epoch_loss = running_loss / len(image_datasets[phase])
        epoch_acc = running_corrects.double() / len(image_datasets[phase])

        print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        if phase == 'Validation' and epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(inceptionv3.state_dict())

    print()

# En iyi model ağırlıklarını yükleme
inceptionv3.load_state_dict(best_model_wts)


Epoch 1/5: 100%|██████████| 229/229 [04:09<00:00,  1.09s/batch, Loss=0.00567]


Epoch 1/5, Loss: 0.0377


Epoch 2/5: 100%|██████████| 229/229 [04:10<00:00,  1.09s/batch, Loss=0.0632]


Epoch 2/5, Loss: 0.0258


Epoch 3/5: 100%|██████████| 229/229 [04:10<00:00,  1.09s/batch, Loss=0.0136]


Epoch 3/5, Loss: 0.0124


Epoch 4/5: 100%|██████████| 229/229 [04:06<00:00,  1.08s/batch, Loss=0.0023]


Epoch 4/5, Loss: 0.0062


Epoch 5/5: 100%|██████████| 229/229 [04:06<00:00,  1.08s/batch, Loss=0.00152]

Epoch 5/5, Loss: 0.0040


In [18]:
correct = 0
total = 0
TP = 0
FP = 0
FN = 0
TN = 0

with torch.no_grad():
    for inputs, labels in dataloaders['Test']:  # dataloaders'ı doğru şekilde kullanın
        inputs, labels = inputs.to(device), labels.to(device)
        outputs, _ = inceptionv3(inputs)  # InceptionV3 modelinden çıktılar alınır
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Kesinlik ve Duyarlılık için TP, FP ve FN değerlerini hesapla
        TP += ((predicted == labels) & (labels == 1)).sum().item()
        FP += ((predicted != labels) & (labels == 0)).sum().item()
        FN += ((predicted != labels) & (labels == 1)).sum().item()
        TN += ((predicted == labels) & (labels == 0)).sum().item()

accuracy = correct / total

# Kesinlik (Precision) hesapla
precision = TP / (TP + FP)

# Duyarlılık (Recall) hesapla
recall = TP / (TP + FN)

# F1-Skoru hesapla
f1_score = 2 * (precision * recall) / (precision + recall)

print(f"Test Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1_score:.4f}")

print(f"TP: {TP:.4f}")
print(f"FP: {FP:.4f}")
print(f"FN: {FN:.4f}")
print(f"TN: {TN:.4f}")


Test Accuracy: 0.5984
Precision: 0.7009
Recall: 0.6192
F1-Score: 0.6575
TP: 239.0000
FP: 102.0000
FN: 147.0000
TN: 132.0000
